# Import Packages

In [1]:
from copy import deepcopy
import sys

sys.path.append('../')

In [2]:
import numpy as np
from tqdm.notebook import tqdm

In [3]:
from datasets import load_data
from src.comparison_methods import MixtureOfMixture
from src.gaussian_mixture import GMMModelSelection
from src.evaluation_metrics import (
    ari,
    f_measure
)
from src.merge_components import (
    EntMergeComponents,
    NEnt1MergeComponents,
    DEMPMergeComponents,
    DEMP2MergeComponents,
    MCMergeComponents,
    NMCMergeComponents
)

# GMM + BIC

In [16]:
def experiment_gmm_bic(data_name, repeat=10):
    
    X, Z_true = load_data(data_name, scaling=True)
    
    def one_trial(seed):
        gmm = GMMModelSelection(K_max=10, mode='GMM_BIC', random_state=seed, tol=1e-3)
        gmm.fit(X)
        Z_pred = gmm.predict(X)
        K_ = gmm.K_
        f_measure_ = f_measure(Z_true, Z_pred)
        ari_ = ari(Z_true, Z_pred)

        return K_, f_measure_, ari_

    K_list = []
    f_measure_list = []
    ari_list = []

    for t in tqdm(range(repeat), leave=False):
        K_, f_measure_, ari_ = one_trial(seed=t)
        K_list.append(K_)
        f_measure_list.append(f_measure_)
        ari_list.append(ari_)

    print('--- average score ---')
    print(f'  K:         {np.mean(K_list)}')
    print(f'  f_measure: {np.mean(f_measure_list)}')
    print(f'  ARI:       {np.mean(ari_list)}')

In [20]:
for data_name in ['AIS', 'beetles', 'crabs', 'DLBCL', 'ecoli', 'seeds', 'wisconsin', 'yeast']:
    print(f'[{data_name}]')
    experiment_gmm_bic(data_name, repeat=10)
    print('')

[AIS]


--- average score ---
  K:         3.0
  f_measure: 0.9123585212839016
  ARI:       0.7434419467778124

[beetles]


--- average score ---
  K:         2.6
  f_measure: 0.8048430417995636
  ARI:       0.6026185376664845

[crabs]


--- average score ---
  K:         3.0
  f_measure: 0.8100491228070176
  ARI:       0.5954506332624884

[DLBCL]


--- average score ---
  K:         6.6
  f_measure: 0.73412256107192
  ARI:       0.5057601013853614

[ecoli]


--- average score ---
  K:         4.0
  f_measure: 0.7872715346245572
  ARI:       0.5896236630278274

[seeds]


--- average score ---
  K:         2.0
  f_measure: 0.7938342967244701
  ARI:       0.5422444052734889

[wisconsin]


--- average score ---
  K:         3.0
  f_measure: 0.8571248492632101
  ARI:       0.6168833014070565

[yeast]


--- average score ---
  K:         3.5
  f_measure: 0.8635955057743191
  ARI:       0.5162198364478092



# GMM + DNML

In [21]:
def experiment_gmm_dnml(data_name, repeat=10):
    
    X, Z_true = load_data(data_name, scaling=True)
    
    def one_trial(seed):
        gmm = GMMModelSelection(K_max=10, mode='GMM_DNML', random_state=seed, tol=1e-3)
        gmm.fit(X)
        Z_pred = gmm.predict(X)
        K_ = gmm.K_
        f_measure_ = f_measure(Z_true, Z_pred)
        ari_ = ari(Z_true, Z_pred)

        return K_, f_measure_, ari_

    K_list = []
    f_measure_list = []
    ari_list = []

    for t in tqdm(range(repeat), leave=False):
        K_, f_measure_, ari_ = one_trial(seed=t)
        K_list.append(K_)
        f_measure_list.append(f_measure_)
        ari_list.append(ari_)

    print('--- average score ---')
    print(f'  K:         {np.mean(K_list)}')
    print(f'  f_measure: {np.mean(f_measure_list)}')
    print(f'  ARI:       {np.mean(ari_list)}')

In [22]:
for data_name in ['AIS', 'beetles', 'crabs', 'DLBCL', 'ecoli', 'seeds', 'wisconsin', 'yeast']:
    print(f'[{data_name}]')
    experiment_gmm_dnml(data_name, repeat=10)
    print('')

[AIS]


--- average score ---
  K:         1.0
  f_measure: 0.6710526315789472
  ARI:       0.0

[beetles]


--- average score ---
  K:         1.0
  f_measure: 0.5904761904761904
  ARI:       0.0

[crabs]


--- average score ---
  K:         1.0
  f_measure: 0.4
  ARI:       0.0

[DLBCL]


--- average score ---
  K:         2.8
  f_measure: 0.9028187107166232
  ARI:       0.8695546900271685

[ecoli]


--- average score ---
  K:         4.0
  f_measure: 0.7872715346245572
  ARI:       0.5896236630278274

[seeds]


--- average score ---
  K:         1.0
  f_measure: 0.5
  ARI:       0.0

[wisconsin]


--- average score ---
  K:         2.0
  f_measure: 0.9142355008787346
  ARI:       0.6851456109359129

[yeast]


--- average score ---
  K:         1.0
  f_measure: 0.8503213957759412
  ARI:       0.0



# MixMix

In [4]:
def experiment_mixmix(data_name, K, L, is_dlbcl, repeat=10):
    
    X, Z_true = load_data(data_name, scaling=True)
    
    def one_trial(seed):
        mixmix = MixtureOfMixture(K=K, L=L, M=4000, burnin=4000, is_dlbcl=is_dlbcl, random_state=seed)
        mixmix.fit(X)
        K_ = mixmix.K_pred_
        Z_pred = mixmix.Z_pred_
        if Z_pred is None:
            Z_pred = np.zeros(len(X), dtype=int)
        f_measure_ = f_measure(Z_true, Z_pred)
        ari_ = ari(Z_true, Z_pred)

        return K_, f_measure_, ari_

    K_list = []
    f_measure_list = []
    ari_list = []

    for t in tqdm(range(repeat), leave=False):
        K_, f_measure_, ari_ = one_trial(seed=t)
        K_list.append(K_)
        f_measure_list.append(f_measure_)
        ari_list.append(ari_)

    print('--- average score ---')
    print(f'  K:         {np.mean(K_list)}')
    print(f'  f_measure: {np.mean(f_measure_list)}')
    print(f'  ARI:       {np.mean(ari_list)}')

In [7]:
for data_name in ['AIS', 'beetles', 'crabs', 'DLBCL', 'ecoli', 'seeds', 'wisconsin', 'yeast']:
    if data_name == 'DLBCL':
        K = 30
        L = 15
        is_dlbcl = True
    else:
        K = 10
        L = 5
        is_dlbcl = False
    print(f'[{data_name}]')
    experiment_mixmix(data_name, K=K, L=L, is_dlbcl=is_dlbcl, repeat=10)
    print('')

[AIS]


--- average score ---
  K:         2.7
  f_measure: 0.9250210267952381
  ARI:       0.751080268051184

[beetles]


--- average score ---
  K:         1.1
  f_measure: 0.6368601404886418
  ARI:       0.1104538206148786

[crabs]


--- average score ---
  K:         1.0
  f_measure: 0.4
  ARI:       0.0

[DLBCL]


--- average score ---
  K:         7.4
  f_measure: 0.7606064523219048
  ARI:       0.5013456399329312

[ecoli]


--- average score ---
  K:         4.3
  f_measure: 0.7935108671940744
  ARI:       0.5487520100902774

[seeds]


--- average score ---
  K:         3.2
  f_measure: 0.8491764300500689
  ARI:       0.622642625008173

[wisconsin]


--- average score ---
  K:         2.1
  f_measure: 0.947312032784007
  ARI:       0.7986272353153039

[yeast]


--- average score ---
  K:         2.7
  f_measure: 0.911456123509837
  ARI:       0.5885335441109534



# Merge Components

In [6]:
def experiment_merge(data_name, algorithm_class, repeat=10):

    X, Z_true = load_data(data_name, scaling=True)
    
    def one_trial(seed):
        gmm = GMMModelSelection(K_max=20, mode='BGMM', weight_concentration_prior=0.1, random_state=seed, tol=1e-3)
        gmm.fit(X)
        prob_0 = gmm.prob_latent(X)

        # delete 0 prob
        prob_0 = np.delete(prob_0, (np.sum(prob_0, axis=0) < 1e-50), axis=1)
        prob_0 = prob_0 / np.sum(prob_0, axis=1).reshape((-1, 1))

        # merge
        algorithm = algorithm_class()
        algorithm.fit(prob_0)

        f_measure_list = []
        ari_list = []
        
        for K in range(1, prob_0.shape[1] + 1):
            Z_pred = np.argmax(algorithm.prob_merged(prob_0, K_merged=K), axis=1)
            ari_list.append(ari(Z_true, Z_pred))
            f_measure_list.append(f_measure(Z_true, Z_pred))

        K_best_f_measure = np.argmax(f_measure_list) + 1
        f_measure_best = max(f_measure_list)
        K_best_ari = np.argmax(ari_list) + 1
        ari_best = max(ari_list)
        K_nmc = algorithm.K_nmc_
        f_measure_nmc = f_measure_list[K_nmc - 1]
        ari_nmc = ari_list[K_nmc - 1]

        return K_best_f_measure, f_measure_best, K_best_ari, ari_best, K_nmc, f_measure_nmc, ari_nmc

    K_best_f_measure_list = []
    f_measure_best_list = []
    K_best_ari_list = []
    ari_best_list = []
    K_nmc_list = []
    f_measure_nmc_list = []
    ari_nmc_list = []

    for t in tqdm(range(repeat), leave=False):
        K_best_f_measure, f_measure_best, K_best_ari, ari_best, K_nmc, f_measure_nmc, ari_nmc = one_trial(seed=t)
        K_best_f_measure_list.append(K_best_f_measure)
        f_measure_best_list.append(f_measure_best)
        K_best_ari_list.append(K_best_ari)
        ari_best_list.append(ari_best)
        K_nmc_list.append(K_nmc)
        f_measure_nmc_list.append(f_measure_nmc)
        ari_nmc_list.append(ari_nmc)

    print('--- average score ---')
    print('  best')
    print(f'    f_measure (K):  {np.mean(f_measure_best_list)} ({np.mean(K_best_f_measure_list)})')
    print(f'    ARI (K):        {np.mean(ari_best_list)} ({np.mean(K_best_ari_list)})')
    print('  NMC')
    print(f'    K:         {np.mean(K_nmc_list)}')
    print(f'    f_measure: {np.mean(f_measure_nmc_list)}')
    print(f'    ARI:       {np.mean(ari_nmc_list)}')

## Ent

In [7]:
for data_name in ['AIS', 'beetles', 'crabs', 'DLBCL', 'ecoli', 'seeds', 'wisconsin', 'yeast']:
    print(f'[{data_name}]')
    experiment_merge(data_name, algorithm_class=EntMergeComponents, repeat=10)
    print('')

[AIS]


--- average score ---
  best
    f_measure (K):  0.9156007492641155 (18.0)
    ARI (K):        0.700465529867522 (19.0)
  NMC
    K:         19.0
    f_measure: 0.8915827870875731
    ARI:       0.700465529867522

[beetles]


--- average score ---
  best
    f_measure (K):  0.9864864864864863 (20.0)
    ARI (K):        0.9577751286738053 (20.0)
  NMC
    K:         20.0
    f_measure: 0.9864864864864863
    ARI:       0.9577751286738053

[crabs]


--- average score ---
  best
    f_measure (K):  0.8655653052611327 (19.1)
    ARI (K):        0.7069945135356266 (19.4)
  NMC
    K:         17.8
    f_measure: 0.821852830112954
    ARI:       0.6264898645018905

[DLBCL]


--- average score ---
  best
    f_measure (K):  0.9305769504935579 (17.6)
    ARI (K):        0.9127728790626739 (17.6)
  NMC
    K:         17.6
    f_measure: 0.9305769504935579
    ARI:       0.9127728790626739

[ecoli]


--- average score ---
  best
    f_measure (K):  0.8738386328508442 (19.0)
    ARI (K):        0.7593004361753439 (19.0)
  NMC
    K:         19.0
    f_measure: 0.8738386328508442
    ARI:       0.7593004361753439

[seeds]


--- average score ---
  best
    f_measure (K):  0.9001411175835157 (19.4)
    ARI (K):        0.7674084622578142 (19.4)
  NMC
    K:         18.1
    f_measure: 0.8517249539537453
    ARI:       0.6573357932829067

[wisconsin]


--- average score ---
  best
    f_measure (K):  0.8973337213961358 (19.2)
    ARI (K):        0.6878726999962621 (19.2)
  NMC
    K:         19.2
    f_measure: 0.8973337213961358
    ARI:       0.6878726999962621

[yeast]


--- average score ---
  best
    f_measure (K):  0.8665975839435079 (18.6)
    ARI (K):        0.508306379559955 (18.6)
  NMC
    K:         18.6
    f_measure: 0.8665975839435079
    ARI:       0.508306379559955



## NEnt1

In [8]:
for data_name in ['AIS', 'beetles', 'crabs', 'DLBCL', 'ecoli', 'seeds', 'wisconsin', 'yeast']:
    print(f'[{data_name}]')
    experiment_merge(data_name, algorithm_class=NEnt1MergeComponents, repeat=10)
    print('')

[AIS]


--- average score ---
  best
    f_measure (K):  0.9009900990099009 (2.0)
    ARI (K):        0.7010754158738216 (3.0)
  NMC
    K:         4.0
    f_measure: 0.8915827870875731
    ARI:       0.700465529867522

[beetles]


--- average score ---
  best
    f_measure (K):  0.9864864864864863 (3.0)
    ARI (K):        0.9577751286738053 (3.0)
  NMC
    K:         3.0
    f_measure: 0.9864864864864863
    ARI:       0.9577751286738053

[crabs]


--- average score ---
  best
    f_measure (K):  0.888837136020566 (5.7)
    ARI (K):        0.7392607708647603 (5.7)
  NMC
    K:         3.6
    f_measure: 0.8276826800382494
    ARI:       0.6417765741406162

[DLBCL]


--- average score ---
  best
    f_measure (K):  0.9217074431770907 (10.2)
    ARI (K):        0.8516782106050738 (3.2)
  NMC
    K:         6.9
    f_measure: 0.9048796947920265
    ARI:       0.833820652289333

[ecoli]


--- average score ---
  best
    f_measure (K):  0.8603257661917023 (8.9)
    ARI (K):        0.718582565760888 (7.0)
  NMC
    K:         4.7
    f_measure: 0.822830022148327
    ARI:       0.6382520061151169

[seeds]


--- average score ---
  best
    f_measure (K):  0.9277171259505603 (6.7)
    ARI (K):        0.8103395252157997 (6.7)
  NMC
    K:         4.9
    f_measure: 0.8215661977789328
    ARI:       0.5941039444059877

[wisconsin]


--- average score ---
  best
    f_measure (K):  0.904137125875317 (5.3)
    ARI (K):        0.6880454633823212 (5.3)
  NMC
    K:         5.6
    f_measure: 0.904137125875317
    ARI:       0.6880430501108991

[yeast]


--- average score ---
  best
    f_measure (K):  0.8681326623305854 (8.8)
    ARI (K):        0.5107536455676865 (8.8)
  NMC
    K:         8.9
    f_measure: 0.867650203440351
    ARI:       0.510706686380105



## DEMP

In [9]:
for data_name in ['AIS', 'beetles', 'crabs', 'DLBCL', 'ecoli', 'seeds', 'wisconsin', 'yeast']:
    print(f'[{data_name}]')
    experiment_merge(data_name, algorithm_class=DEMPMergeComponents, repeat=10)
    print('')

[AIS]


--- average score ---
  best
    f_measure (K):  0.905940594059406 (2.0)
    ARI (K):        0.6660528020133615 (2.4)
  NMC
    K:         4.8
    f_measure: 0.8220002028904956
    ARI:       0.5762624156818488

[beetles]


--- average score ---
  best
    f_measure (K):  0.9864864864864863 (3.0)
    ARI (K):        0.9577751286738053 (3.0)
  NMC
    K:         3.0
    f_measure: 0.9864864864864863
    ARI:       0.9577751286738053

[crabs]


--- average score ---
  best
    f_measure (K):  0.8772142857142857 (4.3)
    ARI (K):        0.7323885283859652 (4.3)
  NMC
    K:         3.3
    f_measure: 0.8226818890546749
    ARI:       0.6403250342004727

[DLBCL]


--- average score ---
  best
    f_measure (K):  0.9519458016762355 (4.9)
    ARI (K):        0.9343119561869788 (4.9)
  NMC
    K:         9.1
    f_measure: 0.7581104137148456
    ARI:       0.522906410166817

[ecoli]


--- average score ---
  best
    f_measure (K):  0.8736702123255304 (6.2)
    ARI (K):        0.7627139409965497 (6.2)
  NMC
    K:         5.7
    f_measure: 0.8672272395519723
    ARI:       0.7537966016553759

[seeds]


--- average score ---
  best
    f_measure (K):  0.908453802228834 (4.7)
    ARI (K):        0.7822831889361379 (4.9)
  NMC
    K:         5.3
    f_measure: 0.8860276811466724
    ARI:       0.7281821553809064

[wisconsin]


--- average score ---
  best
    f_measure (K):  0.9054204814983139 (3.4)
    ARI (K):        0.7338110103818493 (3.6)
  NMC
    K:         4.4
    f_measure: 0.8808457680544798
    ARI:       0.6701758247873809

[yeast]


--- average score ---
  best
    f_measure (K):  0.9418398979593599 (2.3)
    ARI (K):        0.7685689567022634 (2.3)
  NMC
    K:         8.0
    f_measure: 0.8697694985757447
    ARI:       0.5143736018282186



## DEMP2

In [10]:
for data_name in ['AIS', 'beetles', 'crabs', 'DLBCL', 'ecoli', 'seeds', 'wisconsin', 'yeast']:
    print(f'[{data_name}]')
    experiment_merge(data_name, algorithm_class=DEMP2MergeComponents, repeat=10)
    print('')

[AIS]


--- average score ---
  best
    f_measure (K):  0.905940594059406 (2.0)
    ARI (K):        0.6574496200498213 (2.0)
  NMC
    K:         5.0
    f_measure: 0.8046045568412262
    ARI:       0.5452116371354305

[beetles]


--- average score ---
  best
    f_measure (K):  0.9864864864864863 (3.0)
    ARI (K):        0.9577751286738053 (3.0)
  NMC
    K:         3.0
    f_measure: 0.9864864864864863
    ARI:       0.9577751286738053

[crabs]


--- average score ---
  best
    f_measure (K):  0.8767142857142858 (4.3)
    ARI (K):        0.731282072947511 (4.3)
  NMC
    K:         3.3
    f_measure: 0.8221711391406741
    ARI:       0.6387474170073114

[DLBCL]


--- average score ---
  best
    f_measure (K):  0.9521476496465281 (4.9)
    ARI (K):        0.9356818638508984 (5.0)
  NMC
    K:         9.2
    f_measure: 0.7538627585400122
    ARI:       0.5206689280085605

[ecoli]


--- average score ---
  best
    f_measure (K):  0.8753903468339616 (5.7)
    ARI (K):        0.7631941705657869 (6.1)
  NMC
    K:         5.7
    f_measure: 0.8666676452983191
    ARI:       0.7540715691756052

[seeds]


--- average score ---
  best
    f_measure (K):  0.9120735136337232 (4.1)
    ARI (K):        0.787596719742687 (4.5)
  NMC
    K:         5.2
    f_measure: 0.8921910332781617
    ARI:       0.7446941360262593

[wisconsin]


--- average score ---
  best
    f_measure (K):  0.9046052837543475 (2.7)
    ARI (K):        0.7320021911551049 (2.7)
  NMC
    K:         3.6
    f_measure: 0.8801333909565056
    ARI:       0.669698898784691

[yeast]


--- average score ---
  best
    f_measure (K):  0.9442492012779551 (2.0)
    ARI (K):        0.7733499292001705 (2.0)
  NMC
    K:         8.5
    f_measure: 0.8200078196575419
    ARI:       0.40241825800065384



## MC

In [11]:
for data_name in ['AIS', 'beetles', 'crabs', 'DLBCL', 'ecoli', 'seeds', 'wisconsin', 'yeast']:
    print(f'[{data_name}]')
    experiment_merge(data_name, algorithm_class=MCMergeComponents, repeat=10)
    print('')

[AIS]


--- average score ---
  best
    f_measure (K):  0.9306930693069304 (2.0)
    ARI (K):        0.7407112106490014 (2.0)
  NMC
    K:         5.0
    f_measure: 0.802787831653256
    ARI:       0.5341674636233911

[beetles]


--- average score ---
  best
    f_measure (K):  0.9864864864864863 (3.0)
    ARI (K):        0.9577751286738053 (3.0)
  NMC
    K:         3.0
    f_measure: 0.9864864864864863
    ARI:       0.9577751286738053

[crabs]


--- average score ---
  best
    f_measure (K):  0.8627073379707395 (5.6)
    ARI (K):        0.6995578778056432 (5.6)
  NMC
    K:         6.2
    f_measure: 0.8307715189946286
    ARI:       0.6596833697389332

[DLBCL]


--- average score ---
  best
    f_measure (K):  0.9207681141789916 (3.6)
    ARI (K):        0.8492250655262144 (3.6)
  NMC
    K:         11.3
    f_measure: 0.7062058968363178
    ARI:       0.4520415924064037

[ecoli]


--- average score ---
  best
    f_measure (K):  0.870416369280121 (4.9)
    ARI (K):        0.7442290766021828 (4.9)
  NMC
    K:         9.0
    f_measure: 0.8601298864553885
    ARI:       0.7003670841693705

[seeds]


--- average score ---
  best
    f_measure (K):  0.8864726413602609 (7.1)
    ARI (K):        0.745492258822495 (7.1)
  NMC
    K:         8.6
    f_measure: 0.8784795784366353
    ARI:       0.7283862472423367

[wisconsin]


--- average score ---
  best
    f_measure (K):  0.8863439855161376 (2.7)
    ARI (K):        0.6638800411608899 (4.0)
  NMC
    K:         6.1
    f_measure: 0.8575728781944832
    ARI:       0.6327480271340877

[yeast]


--- average score ---
  best
    f_measure (K):  0.9279552715654953 (2.0)
    ARI (K):        0.7091886763002544 (2.0)
  NMC
    K:         9.7
    f_measure: 0.7710800486769616
    ARI:       0.3134260891503135



## NMC

In [12]:
for data_name in ['AIS', 'beetles', 'crabs', 'DLBCL', 'ecoli', 'seeds', 'wisconsin', 'yeast']:
    print(f'[{data_name}]')
    experiment_merge(data_name, algorithm_class=NMCMergeComponents, repeat=10)
    print('')

[AIS]


--- average score ---
  best
    f_measure (K):  0.9156007492641155 (3.0)
    ARI (K):        0.700465529867522 (4.0)
  NMC
    K:         4.0
    f_measure: 0.8915827870875731
    ARI:       0.700465529867522

[beetles]


--- average score ---
  best
    f_measure (K):  0.9864864864864863 (3.0)
    ARI (K):        0.9577751286738053 (3.0)
  NMC
    K:         3.0
    f_measure: 0.9864864864864863
    ARI:       0.9577751286738053

[crabs]


--- average score ---
  best
    f_measure (K):  0.8926972817295932 (4.3)
    ARI (K):        0.7484014414272265 (4.3)
  NMC
    K:         3.1
    f_measure: 0.8283357159431819
    ARI:       0.6432628582612688

[DLBCL]


--- average score ---
  best
    f_measure (K):  0.9487813575460056 (7.1)
    ARI (K):        0.9277175462254874 (7.0)
  NMC
    K:         6.4
    f_measure: 0.9160452515564078
    ARI:       0.87772509232502

[ecoli]


--- average score ---
  best
    f_measure (K):  0.8749076748172093 (6.4)
    ARI (K):        0.7691275778030197 (6.4)
  NMC
    K:         4.3
    f_measure: 0.84829518317622
    ARI:       0.725437496613987

[seeds]


--- average score ---
  best
    f_measure (K):  0.9380490698040246 (4.3)
    ARI (K):        0.8317412488463155 (4.4)
  NMC
    K:         3.2
    f_measure: 0.8095845356878719
    ARI:       0.5749939278709376

[wisconsin]


--- average score ---
  best
    f_measure (K):  0.9449912126537784 (2.0)
    ARI (K):        0.7908557575700376 (2.0)
  NMC
    K:         2.9
    f_measure: 0.9249399222231128
    ARI:       0.7315976115757847

[yeast]


--- average score ---
  best
    f_measure (K):  0.941613556478891 (2.1)
    ARI (K):        0.7604293654630472 (2.1)
  NMC
    K:         6.9
    f_measure: 0.8783098257596113
    ARI:       0.5244202374165469

